In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # regex

In [ ]:
## frequency of parts overall and across sets
inv_parts = pd.read_csv("../data/raw/inventory_parts.csv")
parts = pd.read_csv("../data/raw/parts.csv")

freq_sets = {}
for e in inv_parts.part_num.tolist():
    if e in freq_sets:
        freq_sets[e] += 1
    else:
        freq_sets.update({e: 1})

# todo: convert to df, add column with sum of quantity for each part_num 

In [ ]:
## visualize rise and cease of lego parts over time in a density plot each part_num
# note that processing time takes forever

inv_parts = pd.read_csv("../data/raw/inventory_parts.csv")
inv_fig = pd.read_csv("../data/raw/inventory_minifigs.csv")
parts = pd.read_csv("../data/raw/parts.csv")
sets = pd.read_csv("../data/raw/sets.csv")
inventories = pd.read_csv("../data/raw/inventories.csv")
relation = pd.read_csv("../data/raw/part_relationships.csv")

# replace child_part_num with parent_part_num in inventories df
invParent = inv_parts.copy()
childToParent = pd.Series(relation.parent_part_num.values, index=relation.child_part_num).to_dict()
invParent['part_num'].replace(childToParent, inplace=True)

# eliminate all part_num containing stickers, non-lego (eg booklets),  
partsClean = parts.copy()
childPartNum = []
for p in relation.child_part_num:
    childPartNum.append(partsClean[partsClean['part_num'] == p].index.values[0])
partsClean.drop(childPartNum, inplace=True)
partsClean.drop(partsClean[partsClean['part_cat_id'] == 58].index, inplace=True)
partsClean.drop(partsClean[partsClean['part_cat_id'] == 17].index, inplace=True)

# map the sets which contain part_num to the according year
partsOverTime_sets = pd.DataFrame(columns=['part_num','name']+list(range(1949,2021)))
# loop over all part_num p in inv_parts.part_num to fill in df partsOverTime_sets
for p in partsClean.part_num:
    # idk if good or bad coding style, but it makes me feel safe and helps debugging
    partToId = 0
    idToSet = 0
    setToYear = 0
    figToId = 0
    removeFig = []
    # init part dict with part_num, name and years from 1949 to 2020
    part = {'part_num' : p}
    part.update({'name' : parts[parts['part_num'] == p].iloc[0,1]})
    for y in list(range(1949, 2021)):
        part.update({y: 0})
    # part to inventory id (list)
    partToId = invParent[invParent['part_num'] == p].inventory_id.to_list()
    # remove duplicates from list, since these are parts occuring multiple times due different colors
    partToId = list(dict.fromkeys(partToId)) 
    # inventory id to set num
    idToSet = [inventories[inventories['id'] == e].iloc[0,2] for e in partToId]
    # in case of figures (set_num = fig-...) we need to go to inv_fig to get inv_id (list) -> inventories to get set_num(s) 
    for f in idToSet:
        if re.match(r"fig-[0-9]+", f):
            figToId = inv_fig[inv_fig['fig_num'] == f].inventory_id.to_list()
            removeFig.append(f)
            idToSet.extend([inventories[inventories['id'] == e].iloc[0,2] for e in figToId])
    idToSet = [e for e in idToSet if e not in removeFig]
    # set num to year
    setToYear = [sets[sets['set_num'] == e].iloc[0,2] for e in idToSet]
    # update dict of part with occurences per year
    for e in setToYear:
            part[e] += 1
    # finally append dict part as row to df partsOverTime_sets
    partsOverTime_sets = partsOverTime_sets.append(part, ignore_index=True)

# sum up sets / occurences across all years

# create csv with sets(years) series: part_num, name, year_<min>, ..., year_<max> <- which contain the quantity of sets with part_num in this year
# note that min(year) = 1949 and max(year) = 2020 but only until may
partsOverTime_sets.to_csv("../data/processed/partsOverTime.csv", index = False)

## maybe later:
# map part quantity to the accoring year, note that diffrent colors shall count as same part
# create csv with quantity(years) series: part_num, name, year_<min>, ..., year_<max> <- which contain the quantity overall for part_num in this year

In [ ]:
partsOverTime_sets.describe()